# 5 DATA SET CODIESP

In [ ]:
from nltk.corpus import reuters
import nltk
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from yellowbrick.text import TSNEVisualizer
import warnings
warnings.filterwarnings("ignore")
#import dill
# dill.load_session("assignment2.db")

### TRAIN P

In [ ]:
from pathlib import Path
df_train_clinical_P = pd.read_csv(r"C:\Users\Bluetab\Desktop\TFM\codiesp\final_dataset_v4_to_publish\train\trainP.tsv", header=None, sep="\t")

df_train_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'C:\Users\Bluetab\Desktop\TFM\codiesp\final_dataset_v4_to_publish\train\text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_train_clinical_P)):
    df_2 = load_articles(df_train_clinical_P, i)
    df_train_P = df_train_P.append(df_2)
    count = count + 1


df_train_P = df_train_P.set_axis(np.array(range(0,len(df_train_clinical_P))))
df_train_P[1] = df_train_clinical_P[1]
df_train_P.columns = ['content', 'labels']
df_train_P.head(10)

### TEST P

In [ ]:
from pathlib import Path
df_test_clinical_P = pd.read_csv(r"C:\Users\Bluetab\Desktop\TFM\codiesp\final_dataset_v4_to_publish\test\testP.tsv", header=None, sep="\t")

df_test_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'C:\Users\Bluetab\Desktop\TFM\codiesp\final_dataset_v4_to_publish\test\text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n", dtype='unicode', error_bad_lines=False)
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_test_clinical_P)):
    df_2 = load_articles(df_test_clinical_P, i)
    df_test_P = df_test_P.append(df_2)
    count = count + 1


df_test_P = df_test_P.set_axis(np.array(range(0,len(df_test_clinical_P))))
df_test_P[1] = df_test_clinical_P[1]
df_test_P.columns = ['content', 'labels']
df_test_P.head(10)

## DEV P


In [ ]:
from pathlib import Path
df_dev_clinical_P = pd.read_csv(r"C:\Users\Bluetab\Desktop\TFM\codiesp\final_dataset_v4_to_publish\dev\devP.tsv", header=None, sep="\t")

df_dev_P = pd.DataFrame()
def load_articles(x,i):
    try:
        path = Path(r'C:\Users\Bluetab\Desktop\TFM\codiesp\final_dataset_v4_to_publish\dev\text_files_en') / x[0][i]
        path = str(path) +'.txt'
        df_1 = pd.read_csv(path, header=None, sep="\n", dtype='unicode', error_bad_lines=False)
        df_def = df_1.values[0] 
        for j in range(1, len(df_1)):
            df_def = df_def + df_1.values[j]
        return pd.DataFrame(df_def)
    except:
        print(" ")

count= 0
for i in range(0, len(df_dev_clinical_P)):
    df_2 = load_articles(df_dev_clinical_P, i)
    df_dev_P = df_dev_P.append(df_2)
    count = count + 1


df_dev_P = df_dev_P.set_axis(np.array(range(0,790)))
df_dev_P[1] = df_dev_clinical_P[1]
df_dev_P.columns = ['content', 'labels']
df_dev_P.head(10)

## MATRIX TRANSFORMATION

In [ ]:
new_labels_train = []
new_labels_test = []
new_labels_dev = []
lista = ['content']
for i in range(len(df_train_P['labels'])):
    new_labels_train.append([df_train_P['labels'][i]])
    lista = lista + new_labels_train[i]
for i in range(len(df_test_P['labels'])):
    new_labels_test.append([df_test_P['labels'][i]])
    lista = lista + new_labels_test[i]
for i in range(len(df_dev_P['labels'])):
    new_labels_dev.append([df_dev_P['labels'][i]])
    lista = lista + new_labels_dev[i]

    
####### TRAIN #######
myvec = np.zeros((len(df_train_P.drop_duplicates('content', ignore_index=True)),len(lista)))
auxdf = pd.DataFrame(data = myvec, index=range(len(df_train_P.drop_duplicates('content', ignore_index=True))),columns=range(len(lista)))
auxdf.columns = lista
auxdf = auxdf.loc[:,~auxdf.columns.duplicated()]
print(df_train_P)
df_train_P1 = df_train_P
df_train_P = df_train_P.drop_duplicates('content', ignore_index=True)

auxdf['content'] = df_train_P['content']
for j in range(len(auxdf['content'])):
    for i in range(len(df_train_P1['content'])):
        label = df_train_P1['labels'][i]
        if auxdf['content'][j]==df_train_P1['content'][i]:
            label = df_train_P1['labels'][i]
            auxdf[label][j] = 1 
            
train_matrix = auxdf            


#### TEST #####

myvec1 = np.zeros((len(df_test_P.drop_duplicates('content', ignore_index=True)),len(lista)))
auxdf1 = pd.DataFrame(data = myvec1, index=range(len(df_test_P.drop_duplicates('content', ignore_index=True))),columns=range(len(lista)))
auxdf1.columns = lista
auxdf1 = auxdf1.loc[:,~auxdf1.columns.duplicated()]
df_test_P1 = df_test_P
df_test_P = df_test_P.drop_duplicates('content', ignore_index=True)

auxdf1['content'] = df_test_P['content']

for j in range(len(auxdf1['content'])):
    for i in range(len(df_test_P1['content'])):
        label1 = df_test_P1['labels'][i]
        if auxdf1['content'][j]==df_test_P1['content'][i]:
            label1 = df_test_P1['labels'][i]
            auxdf1[label1][j] = 1 
            
test_matrix = auxdf1            


###### DEV ######
myvec3 = np.zeros((len(df_dev_P.drop_duplicates('content', ignore_index=True)),len(lista)))
auxdf3 = pd.DataFrame(data = myvec3, index=range(len(df_dev_P.drop_duplicates('content', ignore_index=True))),columns=range(len(lista)))
auxdf3.columns = lista
auxdf3 = auxdf3.loc[:,~auxdf3.columns.duplicated()]
df_dev_P1 = df_dev_P
df_dev_P = df_dev_P.drop_duplicates('content', ignore_index=True)

auxdf3['content'] = df_dev_P['content']
print(df_dev_P1)
for j in range(len(auxdf3['content'])):
    for i in range(len(df_dev_P1['content'])):
        label3 = df_dev_P1['labels'][i]
        if auxdf3['content'][j]==df_dev_P1['content'][i]:
            label3 = df_dev_P1['labels'][i]
            auxdf3[label3][j] = 1 
            
dev_matrix = auxdf3            

##### TRAIN MATRIX #####
df_train_clinical = pd.DataFrame()
train_matrix = pd.concat([train_matrix, dev_matrix], ignore_index=True)


In [ ]:
train_matrix1 = train_matrix
test_matrix1 = test_matrix

In [ ]:
total = []
for i in range(train_matrix1.shape[1]):
    if i>0:
        total.append(sum(train_matrix1.iloc[:,i]))


total_matrix = pd.concat([train_matrix1, test_matrix1], ignore_index=True)
lista = total_matrix.columns
lista = lista.delete(0)

mat = pd.DataFrame(total, index = lista)
mat = mat.sort_values(0, ascending = False)
print(mat.iloc[:45])
#maxi = len(lista) # coger todas
maxi = 45 # coger 10 o más apariciones. Usar print de total para ver hasta donde coger

coger = mat.iloc[:maxi]
coger = coger.index
coger = coger.insert(0,'content')

train_matrix = train_matrix[coger]
test_matrix = test_matrix[coger]
print(train_matrix)
# print(test_matrix)

In [ ]:
docs_train = 434
docs_dev = 220 #654-434
docs_test = 222

train_words = []
dev_words = []
test_words = []

for i in range(len(train_matrix['content'])):
    if i<434:
        train_words.append(len(train_matrix['content'][i].strip().split(" ")))
    else:
        dev_words.append(len(train_matrix['content'][i].strip().split(" ")))

for i in range(len(test_matrix['content'])):
    test_words.append(len(test_matrix['content'][i].strip().split(" ")))

print('Number of words in train: ',sum(train_words))
print('Number of words in dev: ', sum(dev_words))
print('Number of words in test: ', sum(test_words))

print('\nAverage of words in train: ', round(sum(train_words)/docs_train,2))
print('Average of words in dev: ', round(sum(dev_words)/docs_dev,2))
print('Average of words in test: ', round(sum(test_words)/docs_test,2))

print('\nStandard deviation of words in train: ', round(np.std(train_words),2))
print('Standard deviation of words in dev: ', round(np.std(dev_words),2))
print('Standard deviation of words in test: ', round(np.std(test_words),2))


In [ ]:
a = df_train_P1["labels"].value_counts()
b = 0
print(a)
for i in range(len(a)):
    if a[i] == 1 or a[i] == 2:
        b += 1
        
print(b*100/len(a))
print(b)

In [ ]:
df_test = df_test_P
df_train = df_train_P
s = df_test_P["labels"]
print(s.value_counts())
s.value_counts().plot(kind='bar')
plt.show()

## PRE-PROCESSING

In [ ]:
from nltk.corpus import stopwords
wordnet_lemmatizer = WordNetLemmatizer() # resumir?
stemmer = PorterStemmer() # sacar la raiz de las palabras
stopwords = set(stopwords.words('english')) # creo que es borrar las palabras de I, you, he...

def tokenize_lemma_stopwords(text):
    text = text.replace("\n", " ")
    # split string into words (tokens)
    tokens = nltk.tokenize.word_tokenize(text.lower())
    # keep strings with only alphabets (remove ages)
#     tokens = [t for t in tokens if t.isalpha()]
    # put words into base form
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] 
    tokens = [stemmer.stem(t) for t in tokens]
    # remove short words, they're probably not useful
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    cleanedText = " ".join(tokens)
    return cleanedText

def dataCleaning(df):
    data = df.copy()
    data["content"] =data["content"].apply(tokenize_lemma_stopwords)
    return data
cleanedTrainData = dataCleaning(train_matrix)
cleanedTestData = dataCleaning(test_matrix)

## TF-ID

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

vectorizer = TfidfVectorizer()
vectorised_train_documents = vectorizer.fit_transform(cleanedTrainData["content"]) #learn vocabulary and idf, return document-term matrix
print(vectorised_train_documents)
vectorised_test_documents = vectorizer.transform(cleanedTestData["content"]) # only transform the documents to docuemnt-term matrix
print(vectorised_test_documents)


## YellowBrick

In [ ]:
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.text import UMAPVisualizer
import umap.umap_ as umap

features = vectorizer.get_feature_names() #mapping from feature integer indeces to feature name
visualizer = FreqDistVisualizer(features=features, orient='v', n=30)
visualizer.fit(vectorised_train_documents)
visualizer.ax.tick_params(axis = 'x',labelrotation=80, labelsize = 16)
visualizer.ax.tick_params(axis = 'y', labelsize = 14)
plt.savefig('plot_word_distribution_P_english.png', dpi=300, bbox_inches='tight')
visualizer.show()

umap = UMAPVisualizer(metric="cosine") # Uniform Manifold Approximation and Projecytion (clusters can be seen and no correlation)
umap.fit(vectorised_train_documents)
umap.ax.tick_params(axis = 'x', labelsize = 16)
umap.ax.tick_params(axis = 'y', labelsize = 16)
plt.savefig('plot_umap_P_english.png', dpi=300, bbox_inches='tight')
umap.show()

## ML

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss, average_precision_score
ModelsPerformance= {}

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss


knnClf = KNeighborsClassifier()

knnClf.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
knnPredictions = knnClf.predict(vectorised_test_documents)
# print(knnPredictions.shape)
# print(test_matrix.iloc[:,1:].shape)
macro_f1 = f1_score(test_matrix.iloc[:,1:], knnPredictions, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], knnPredictions, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), knnPredictions.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], knnPredictions)
ModelsPerformance['KNN'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

### Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtClassifier = DecisionTreeClassifier()
dtClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
dtPreds = dtClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], dtPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], dtPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), dtPreds.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], dtPreds)
ModelsPerformance['Decision Tree'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfClassifier = RandomForestClassifier(n_jobs=-1)
rfClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
rfPreds = rfClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], rfPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], rfPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), rfPreds.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], rfPreds)
ModelsPerformance['RF'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

In [ ]:
def pretty_dict(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))
            
pretty_dict(ModelsPerformance)

### Baggings

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagClassifier = OneVsRestClassifier(BaggingClassifier(n_jobs=-1))
bagClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
bagPreds = bagClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], bagPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], bagPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), bagPreds.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], bagPreds)
ModelsPerformance['Bagging'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

In [ ]:
bagClassifier.get_params()

### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

boostClassifier = OneVsRestClassifier(GradientBoostingClassifier())
boostClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
boostPreds = boostClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], boostPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], boostPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), boostPreds.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], boostPreds)
ModelsPerformance['Gradient Boosting'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

### Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

svmClassifier = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
svmClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
svmPreds = svmClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], svmPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], svmPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), svmPreds.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], svmPreds)
ModelsPerformance['Linear SVC'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

In [ ]:
pretty_dict(ModelsPerformance)

## Gradient boosting tunning

In [ ]:
import time
import optuna
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
min_n_estimators = 2
max_n_estimators = 50

max_depth = 25

max_splits=2
min_splits=40

budget=12
def objective_gb(trial):
    
    n_estimators_suggest = trial.suggest_int("n_estimators", min_n_estimators, max_n_estimators)
    
    max_depth_suggest = trial.suggest_int("max_depth",2,max_depth)
    
#     min_samples_split_suggest = trial.suggest_int("min_samples_split", max_splits,min_splits)
    
    loss_suggest=trial.suggest_categorical("loss", [ 'deviance', 'exponential'])
    
    criterion_suggest = trial.suggest_categorical("criterion",['friedman_mse', 'mse', 'mae'])
    
#     imputer_gb_opt = SimpleImputer(strategy = strategy_suggest)
    
    gb_opt = OneVsRestClassifier(GradientBoostingClassifier(max_depth=max_depth_suggest,
#                                         min_samples_split=min_samples_split_suggest,
                                        n_estimators = n_estimators_suggest,
                                        criterion=criterion_suggest,
                                        loss = loss_suggest))
    
    gb_opt.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
    boostPreds = gb_opt.predict(vectorised_test_documents)
    micro_f1 = f1_score(test_matrix.iloc[:,1:], boostPreds, average='micro')
    return micro_f1

start=time.time()
sampler=optuna.samplers.TPESampler(seed=0)
study_gb = optuna.create_study(direction="maximize",sampler=sampler)
np.random.seed(10)
study_gb.optimize(objective_gb, n_trials=budget)
end=time.time()
time_gb_opt=end-start

ModelsPerformance['GB_optuna'] = ['f1_micro', study_gb.best_value], ['params', study_gb.best_params]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

params = study_bgg.best_params

boostClassifier = OneVsRestClassifier(GradientBoostingClassifier(**params))
boostClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
boostPreds = boostClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], boostPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], boostPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), boostPreds.reshape((-1)), average='macro')
ap_micro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), boostPreds.reshape((-1)), average='micro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], boostPreds)
ModelsPerformance['Gradient Boosting'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap_micro", ap_micro], ["ap_macro", ap_macro]
print(ModelsPerformance)

In [ ]:
pretty_dict(ModelsPerformance)

In [ ]:
import time
import optuna
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
min_n_estimators = 2
max_n_estimators = 50

max_feature = 25

max_splits=2
min_splits=40

budget=12
def objective_bgg(trial):
    
    n_estimators_suggest = trial.suggest_int("n_estimators", min_n_estimators, max_n_estimators)
    
#     max_feature_suggest = trial.suggest_int("max_feature",2,max_feature)
    
#     min_samples_split_suggest = trial.suggest_int("min_samples_split", max_splits,min_splits)
    
    bootstrap_suggest=trial.suggest_categorical("bootstrap", [True,False])
    
#     criterion_suggest = trial.suggest_categorical("criterion",['friedman_mse', 'mse', 'mae'])
    
#     imputer_gb_opt = SimpleImputer(strategy = strategy_suggest)
    
    bging_opt = OneVsRestClassifier(BaggingClassifier(
#         max_features=max_feature_suggest,
#                                         min_samples_split=min_samples_split_suggest,
                                        n_estimators = n_estimators_suggest,
                                        bootstrap=bootstrap_suggest
                                        ))
    
    bging_opt.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
    bagPreds = bging_opt.predict(vectorised_test_documents)
    micro_f1 = f1_score(test_matrix.iloc[:,1:], bagPreds, average='micro')
    return micro_f1

start=time.time()
sampler=optuna.samplers.TPESampler(seed=0)
study_bgg = optuna.create_study(direction="maximize",sampler=sampler)
np.random.seed(0)
study_bgg.optimize(objective_bgg, n_trials=budget)
end=time.time()
time_gb_opt=end-start

ModelsPerformance['Bagg_optuna'] = ['f1_micro', study_bgg.best_value], ['params', study_bgg.best_params]


In [ ]:
from sklearn.ensemble import BaggingClassifier

params = study_bgg.best_params
bagClassifier = OneVsRestClassifier(BaggingClassifier(**params))
bagClassifier.fit(vectorised_train_documents, train_matrix.iloc[:,1:])
bagPreds = bagClassifier.predict(vectorised_test_documents)

macro_f1 = f1_score(test_matrix.iloc[:,1:], bagPreds, average='macro')
micro_f1 = f1_score(test_matrix.iloc[:,1:], bagPreds, average='micro')
ap_macro = average_precision_score(test_matrix.iloc[:,1:].to_numpy().reshape((-1)), bagPreds.reshape((-1)), average='macro')
hamLoss = hamming_loss(test_matrix.iloc[:,1:], bagPreds)
ModelsPerformance['Bagging_optuna'] = ["micro_f1", micro_f1], ["macro_f1", macro_f1], ["hamloss", hamLoss], ["ap", ap_macro]
print(ModelsPerformance)

In [ ]:
pretty_dict(ModelsPerformance)